In [1]:
# we need to do this on colabatory because their version of keras that's installed is old
!pip install keras --upgrade
!pip install git+https://github.com/AI-Guru/ngdlm.git

from keras import models, layers, optimizers
from ngdlm import models as ngdlmodels
from ngdlm import utils as ngdlmutils
from keras.datasets import mnist
import numpy as np

Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (2.2.4)
  Cloning https://github.com/AI-Guru/ngdlm.git to /tmp/pip-req-build-asv1f7ab
  Running setup.py bdist_wheel for ngdlm ... - done
  Stored in directory: /tmp/pip-ephem-wheel-cache-svcp6ezf/wheels/93/06/27/e156acb49f475c364c3c9fa4ad4ab7bfa38808bff5bf9c4647
Successfully built ngdlm


Using TensorFlow backend.


In [3]:
# this is the dimensionality of our latent space. We set it randomly to 8 to begin with
latent_dim = 8

# create a base model to begin
# it's a little neural network that takes some input data and puts it into latent space, of size latent_dim
base_input = layers.Input(shape=(28,28))
base_output = base_input
base_output = layers.Flatten()(base_output)
base_output = layers.Dense(512,activation='relu')(base_output)
base_output = layers.Dense(256,activation='relu')(base_output)
base_output = layers.Dense(128,activation='relu')(base_output)
base_output = layers.Dense(latent_dim)(base_output) # no activation means that it will be linear
base = models.Model(base_input,base_output)
base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 28, 28)            0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               401920    
_________________________________________________________________
dense_6 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_7 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_8 (Dense)              (None, 8)                 1032      
Total params: 567,176
Trainable params: 567,176
Non-trainable params: 0
_________________________________________________________________


In [4]:
# this now connects our base model to the 3 triplet loss models
# we have three inputs (images), puts it into our base model, and then outputs into a concatenated latent space
tl = ngdlmodels.TL(base)
tl.summary()

Basemodel:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 28, 28)            0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               401920    
_________________________________________________________________
dense_6 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_7 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_8 (Dense)              (None, 8)                 1032      
Total params: 567,176
Trainable params: 567,176
Non-trainable params: 0
___________________________________________________________

In [0]:
# now we can compile the model
tl.compile(
    optimizer = 'rmsprop',
    triplet_loss = 'euclidean' # this is just straight distances between points in latent space (cosine distance is an alternative) 
)

In [0]:
# load some data for our model
(x_input_train,y_output_train),(x_input_test,y_output_test) = mnist.load_data()
x_input_train = x_input_train.astype('float32') / 255.0
x_input_test = x_input_test.astype('float32') / 255.0

In [0]:
# now we fit our model on mnist training data
history = tl.fit(
    x_input_train,y_output_train,
    epochs = 1000,
    batch_size = 128,
    steps_per_epoch = 1000,
    minibatch_size = 10,
    shuffle = True,
    validation_data = (x_input_test,y_output_test),
    validation_steps = 500
)

Epoch 1/1000...
loss [0.0016720831571146845]
val_loss [0.23433858156204224]
Epoch 2/1000...
loss [0.002009627615334466]
val_loss [0.33618417382240295]
Epoch 3/1000...
loss [0.001660420143045485]
val_loss [0.3056345582008362]
Epoch 4/1000...
loss [0.001547861116239801]
val_loss [0.37241530418395996]
Epoch 5/1000...
loss [0.0016469977933447808]
val_loss [0.3387838304042816]
Epoch 6/1000...
loss [0.0013707638517953455]
val_loss [0.43914666771888733]
Epoch 7/1000...
loss [0.0016662691016681493]
val_loss [0.2714526951313019]
Epoch 8/1000...
loss [0.0013302800415549427]
val_loss [0.3056591749191284]
Epoch 9/1000...
loss [0.0012442319870460778]
val_loss [0.3001728653907776]
Epoch 10/1000...
loss [0.0011684680886100978]
val_loss [0.49239349365234375]
Epoch 11/1000...
loss [0.0012558576432056725]
val_loss [0.26396065950393677]
Epoch 12/1000...
loss [0.001152068650117144]
val_loss [0.30153581500053406]
Epoch 13/1000...
loss [0.0010593898063525557]
val_loss [0.2183527797460556]
Epoch 14/1000...
l